In [ ]:
import numpy as np
import scipy as sp

# Diagonal Format (DIA)

- very simple scheme
- diagonals in dense NumPy array of shape `(n_diag, length)`
  - fixed length -> waste space a bit when far from main diagonal
  - subclass of {class}`_data_matrix` (sparse array classes with
    `.data` attribute)
- offset for each diagonal
  - 0 is the main diagonal
  - negative offset = below
  - positive offset = above
- fast matrix \* vector (sparsetools)
- fast and easy item-wise operations
  - manipulate data array directly (fast NumPy machinery)
- constructor accepts:
  - dense array/matrix
  - sparse array/matrix
  - shape tuple (create empty array)
  - `(data, offsets)` tuple
- no slicing, no individual item access
- use:
  - rather specialized
  - solving PDEs by finite differences
  - with an iterative solver

## Examples

### Create some DIA arrays:

In [ ]:
data = np.array([[1, 2, 3, 4]]).repeat(3, axis=0)
data

In [ ]:
offsets = np.array([0, -1, 2])
mtx = sp.sparse.dia_array((data, offsets), shape=(4, 4))
mtx

In [ ]:
mtx.toarray()

In [ ]:
data = np.arange(12).reshape((3, 4)) + 1
data

In [ ]:
mtx = sp.sparse.dia_array((data, offsets), shape=(4, 4))
mtx.data

In [ ]:
mtx.offsets

In [ ]:
print(mtx)

In [ ]:
mtx.toarray()

### Explanation with a scheme:

```
offset: row

    2:  9
    1:  --10------
    0:  1  . 11  .
    -1:  5  2  . 12
    -2:  .  6  3  .
    -3:  .  .  7  4
        ---------8
```

### Matrix-vector multiplication

In [ ]:
vec = np.ones((4, ))
vec

In [ ]:
mtx @ vec

In [ ]:
(mtx * vec).toarray()